In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\\Users\\Trey\\Desktop\\Capstone II\\Data\\winedata.csv')
df.head()

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.0,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
# remove duplicate rows
df = df.loc[df.duplicated(subset=['description', 'title'], keep='first') == False]
df.reset_index(drop=True, inplace=True)

## Goal is to turn price into a categorical variable so classification models can be used

In [4]:
''' Price will be changed into one of five variables: cheap, below average, average, above average, expensive '''

# new column
df['priceCategory'] = df['price']

# get quantile information
desc = df.price.describe()

# set quantiles
smallest = desc[3]
tenth = np.quantile(df.price, .1)
twentyfifth = desc[4]
seventyfifth = desc[6]
ninedieth = np.quantile(df.price, .9)
largest = desc[7]

In [5]:
desc

count    119928.000000
mean         35.582458
std          40.613389
min           4.000000
25%          18.000000
50%          28.000000
75%          40.000000
max        3300.000000
Name: price, dtype: float64

In [6]:
# change price based on where it lies in regards to quantile
for i in range(0, len(df.priceCategory)):
    if tenth > df.priceCategory[i] >= smallest:
        df.priceCategory[i] = 'cheap'
    elif twentyfifth > df.priceCategory[i] >= tenth:
        df.priceCategory[i] = 'below average'
    elif seventyfifth > df.priceCategory[i] >= twentyfifth:
        df.priceCategory[i] = 'average'
    elif ninedieth > df.priceCategory[i] >= seventyfifth:
        df.priceCategory[i] = 'above average'
    else:
        df.priceCategory[i] = 'expensive'

<ipython-input-6-795bfec07669>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.priceCategory[i] = 'average'
C:\Users\Trey\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-6-795bfec07669>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.priceCategory[i] = 'below average'
<ipython-input-6-795bfec07669>:12: SettingWithCop

In [7]:
#check work
df.priceCategory.value_counts()

average          57902
above average    20667
below average    17830
expensive        12046
cheap            11483
Name: priceCategory, dtype: int64

## Removal of Redundant Columns

In [8]:
# Each taster uses only one twitter so there is no additional information that can be offered with twitter name for models
# the disparity is explained by unkown twitter handles/tasters

print('Number of difference tasters: ', df['taster_name'].nunique())
print('Number of different taster_twitters: ', df['taster_twitter_handle'].nunique(), '\n')
print(df.groupby('taster_name')['taster_twitter_handle'].nunique())


Number of difference tasters:  20
Number of different taster_twitters:  16 

taster_name
Alexander Peartree    1
Anna Lee C. Iijima    1
Anne Krebiehl MW      1
Carrie Dykes          1
Christina Pickard     1
Fiona Adams           1
Jeff Jenssen          1
Jim Gordon            1
Joe Czerwinski        1
Kerin O’Keefe         1
Lauren Buzzeo         1
Matt Kettmann         1
Michael Schachner     1
Mike DeSimone         1
Paul Gregutt          1
Roger Voss            1
Sean P. Sullivan      1
Susan Kostrzewa       1
Virginie Boone        1
unknown               1
Name: taster_twitter_handle, dtype: int64


In [9]:
# drop twitter and price
df.drop(columns=['taster_twitter_handle', 'price'], inplace=True)

In [10]:
# Drop title because it is arguebly the same as unique rowsprint('Number of difference titles: ', df['title'].nunique())
print('Number of difference rows: ', len(df))
print('% difference is:', 100*(df['title'].nunique() - len(df))/len(df),'%')

Number of difference rows:  119928
% difference is: -0.9572410112734308 %


In [11]:
# Drop description because it is arguebly the same as unique rows
print('Number of difference titles: ', df['description'].nunique())
print('Number of difference rows: ', len(df))
print('% difference is:', 100*(df['description'].nunique() - len(df))/len(df),'%')

Number of difference titles:  119895
Number of difference rows:  119928
% difference is: -0.027516509905943565 %


In [12]:
df.drop(columns=['description', 'title'], inplace=True)

In [13]:
# last check
df.nunique()

country             43
designation      37955
points              21
province           425
region_1          1230
taster_name         20
variety            701
winery           16745
priceCategory        5
dtype: int64

In [14]:
# one hot encoding and train test split will be done in the next step
df.to_csv('C:\\Users\\Trey\\Desktop\\Capstone II\\Data\\winedatapre.csv', index=False)